In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 6章 集約
## 6-1 データ全体の集約と代表的な集約関数
### Q：予約履歴の各種集計値の算出
#### Awesome

In [ ]:
%%bigquery
select
    -- （2）集計値の計算
    count(*) as reservation_cnt,          -- カウント
    sum(total_price) as sales,           -- 総和
    avg(total_price) as mean_sales,      -- 平均値
    min(total_price) as min_sales,       -- 最小値
    max(total_price) as max_sales,       -- 最大値
    variance(total_price) as var_sales,  -- 不偏分散
    stddev(total_price) as std_sales    -- 不偏標準偏差
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"

### Q：予約顧客のユニークカウントの算出
#### Awesome

In [ ]:
%%bigquery
select count(distinct customer_id) as customer_cnt
-- （2）ユニークカウントの計算
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"

#### Awesome

In [ ]:
%%bigquery
select approx_count_distinct(customer_id) as customer_cnt
-- （2）ユニークカウントの近似計算
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"

### Q：予約単価の中央値およびパーセンタイル値の算出
#### Awesome

In [ ]:
%%bigquery
select
    -- （3）percentile_contで計算した中央値、パーセンタイル値の取得
    any_value(median_sales) as median_sales,
    any_value(p25_sales) as p25_sales,
    any_value(p75_sales) as p75_sales
from (
    select
        -- （2）中央値、パーセンタイル値の計算
        percentile_cont(total_price, 0.5) over () as median_sales,
        percentile_cont(total_price, 0.25) over () as p25_sales,
        percentile_cont(total_price, 0.75) over () as p75_sales
    from example.reservation
    -- （1）キャンセル済みではない予約の抽出
    where status != "canceled"
)

#### Awesome

In [ ]:
%%bigquery
select
    -- （2）中央値、パーセンタイル値の近似計算
    approx_quantiles(total_price, 100)[offset(50)] as median_sales,
    approx_quantiles(total_price, 100)[offset(25)] as p25_sales,
    approx_quantiles(total_price, 100)[offset(75)] as p75_sales
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"

### Q：ホテルごとの宿泊人数の最頻値の算出
#### Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    people_num as mode_people_num
from (
    select
        hotel_id,
        people_num,
        -- （3）同じhotel_idの中で、people_numの値のカウント順に順位を付与
        rank() over (partition by hotel_id order by count(*) desc) as rn
    from example.reservation
    -- （1）キャンセル済みではない予約の抽出
    where status != "canceled"
    -- （2）hotel_idとpeople_numをキーとしてグループ集約
    group by hotel_id, people_num
)
-- （4）順位の1番のみ抽出
where rn = 1

#### Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    -- （3）people_numの最頻値を取得
    approx_top_count(people_num, 1)[offset(0)].value as mode_people_num
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"
-- （2）hotel_idをキーとしてグループ集約
group by hotel_id

## 6-2 グループごとの集約
### Q：ホテルごとの売上の集計
#### Not Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    -- （3）hotel_idの値ごとにtotal_priceの総和を計算
    sum(total_price) as sales
from (
    select *
    from example.reservation
    -- （1）キャンセル済みではない予約の抽出
    where status != "canceled"
)
-- （2）hotel_idを集約キーとしてgroup by
group by hotel_id

#### Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    -- （3）hotel_idの値ごとにtotal_priceの総和を計算
    sum(total_price) as sales
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"
-- （2）hotel_idを集約キーとしてgroup by
group by hotel_id

### Q：ホテルごと・顧客ごとの予約数の集計
#### Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    customer_id,
    -- （3）hotel_idとcustomer_idの値の組ごとにデータ数をカウント
    count(*)
from example.reservation
-- （1）キャンセル済みではない予約の抽出
where status != "canceled"
-- （2）hotel_idとcustomer_idを集約キーとしてgroup by
group by
    hotel_id,
    customer_id

## 6-3 数値の区間ごとの集約
### Q：等間隔の価格帯ごとにホテル数を集計
#### Awesome

In [ ]:
%%bigquery
select
    -- （1）unit_priceを数値区間に丸めた列を作成
    cast(floor(unit_price / 5000) * 5000 as int) as unit_price_range,
    -- （2）unit_price_rangeの値ごとにデータ数をカウント
    count(*)
from example.hotel
-- （3）unit_price_rangeを集約キーとしてgroup by
group by unit_price_range

### Q：非等間隔の価格帯ごとにホテル数を集計
#### Awesome

In [ ]:
%%bigquery
select
    -- （1）unit_priceを数値区間に丸めた列を作成
    case
        when unit_price < 5000 then "1: 0~4999"
        when unit_price < 10000 then "2: 5000~9999"
        when unit_price < 20000 then "3: 10000~19999"
        when unit_price < 30000 then "4: 20000~29999"
        else "5: 30000~"
    end as unit_price_range,
    -- （2）unit_price_rangeの値ごとにデータ数をカウント
    count(*)
from example.hotel
-- （3）unit_price_rangeを集約キーとしてgroup by
group by unit_price_range

## 6-4 時間の区間ごとの集約
### Q：月ごとの売上の集計
#### Awesome

In [ ]:
%%bigquery
select
    -- （1）checkout_dateの年月部分を取り出した文字列を作成
    format_timestamp("%Y-%m", checkout_date, "Asia/Tokyo") as month,
    -- （2）monthの値ごとにtotal_priceの総和を計算
    sum(total_price) as sales
from example.reservation
-- （3）キャンセル済みではない予約の抽出
where status != "canceled"
-- （4）monthを集約キーとしてgroup by
group by month

## 6-5 条件を満たす行の存在判定
### Q：チェックインの7日以内に予約をキャンセルしたことがある顧客の判定
#### Awesome

In [ ]:
%%bigquery
select
    customer_id,
    --（2）customer_idの値ごとに、キャンセル済み、かつcanceled_atがcheckin_dateの7日以内、
    --    というデータが1件以上存在するかどうかを集計
    max(status = "canceled" and timestamp_diff(checkin_date, canceled_at, day) <= 7)
        as is_canceled_within_7days_to_checkin
from example.reservation
--（1）customer_idを集約キーとしてgroup by
group by customer_id

## 6-6 条件を満たす行のみの集約
### Q：顧客ごとの売上とキャンセル率の算出


#### Awesome

In [ ]:
%%bigquery
select
    customer_id,
    --（2）customer_idの値ごとに、未キャンセルのデータに限定してtotal_priceの総和を計算
    sum(case when status != "canceled" then total_price else 0 end) as total_price,
    --（3）customer_idの値ごとに、キャンセル済みのデータに限定したカウントを全データ数で除算
    cast(count(case when status = "canceled" then 1 end) as float64) / count(*)
        as cancel_rate
from example.reservation
--（1）customer_idを集約キーとしてgroup by
group by customer_id